In [18]:
import numpy as np
import scipy.io as io
import random
import scipy

from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from utils import *
from eeg_feature_extraction.extract_eeg_features import *

In [2]:
#NOTE: We are now using only EEG frequencies for TRT to extract features (maybe, we should also use GD, GPT, or FFD)
def get_eeg_freqs(task:str, sbj:int, freq_domain:str, merge:str):
    """
        Args: Task (NR vs. TSR), Test subject number, EEG frequency domain (theta, alpha, beta, gamma), Binning strategy
        Return: NumPy matrix of respective EEG features on word level
    """
    files = get_matfiles(task)
    data = io.loadmat(files[sbj], squeeze_me=True, struct_as_record=False)['sentenceData']
    n_words = sum([len(sent.word) for sent in data if not isinstance(sent.word, float)])
    n_electrodes = 105
    word2eeg = np.zeros((n_words, n_electrodes))
    
    if freq_domain == 'theta':
        fields = ['TRT_t1', 'TRT_t2']
    elif freq_domain == 'alpha':
        fields = ['TRT_a1', 'TRT_a2']
    elif freq_domain == 'beta':
        fields = ['TRT_b1', 'TRT_b2']    
    elif freq_domain == 'gamma':
        fields = ['TRT_g1', 'TRT_g2']
        
    fixated = 0
    for sent in data:
        # if there is no data, skip sentence (most probably due to technical issues)
        if isinstance(sent.word, float):
            continue
        else:
            for word in sent.word:
                # if there was no fixation, skip word
                if isinstance(word.nFixations, np.ndarray):
                    continue
                else:
                    if merge == 'avg':
                        eeg_freq = np.mean(np.vstack([getattr(word, field) if hasattr(word, field) and len(getattr(word, field)) 
                                                      > 0 else 0 for field in fields]), axis = 0)
                    elif merge == 'max':
                        eeg_freq = np.amax(np.vstack([getattr(word, field) if hasattr(word, field) and len(getattr(word, field))
                                                      > 0 else 0 for field in fields]), axis = 0)
                    else:
                        raise ValueError('Binning strategy must be one of {max-pool, average}')
                    eeg_freq[np.isnan(eeg_freq)] = 0
                    word2eeg[fixated] += eeg_freq
                    fixated += 1
    word2eeg = word2eeg[:fixated, :]
    return word2eeg

In [3]:
def mean_freq_per_sbj(task:str, freq_domain:str, merge:str):
    sbjs_to_skip = [6, 11] if task == 'task2' else [3, 7, 11]
    X = []
    for i in range(12):
        if i not in sbjs_to_skip:
            X.append(get_eeg_freqs(task, i, freq_domain, merge))
    X_mean = np.zeros((X[0].shape[0], 105))
    if task == 'task2':
        D_0, D_1, D_2, D_3, D_4, D_5, D_7, D_8, D_9, D_10 = X 
        for i, (sbj_0, sbj_1, sbj_2, sbj_3, sbj_4, sbj_5, sbj_7, sbj_8, sbj_9, sbj_10) in enumerate(zip(D_0, D_1, D_2, D_3, D_4, D_5, D_7, D_8, D_9, D_10)):
            X_mean[i] += np.mean((sbj_0, sbj_1, sbj_2, sbj_3, sbj_4, sbj_5, sbj_7, sbj_8, sbj_9, sbj_10), axis=0)
    elif task == 'task3':
        D_0, D_1, D_2, D_4, D_5, D_6, D_8, D_9, D_10 = X
        for i, (sbj_0, sbj_1, sbj_2, sbj_4, sbj_5, sbj_6, sbj_8, sbj_9, sbj_10) in enumerate(zip(D_0, D_1, D_2, D_4, D_5, D_6, D_8, D_9, D_10)):
             X_mean[i] += np.mean((sbj_0, sbj_1, sbj_2, sbj_4, sbj_5, sbj_6, sbj_8, sbj_9, sbj_10), axis=0)
    return X_mean

In [4]:
def clf_fit(X_train, X_test, y_train, y_test, clf, rnd_state=42):
    if clf == 'RandomForest':
        model = RandomForestClassifier(n_estimators=100, criterion='gini', bootstrap=False, random_state=rnd_state)
    elif clf == 'LogReg':
        model = LogisticRegressionCV(cv=5, max_iter=1000, random_state=rnd_state,)
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    #print(model.score(X_test, y_test))
    print(accuracy_score(y_test, y_hat))
    if clf == 'LogReg':
        return model.coef_
    else:
        return model.feature_importances_

In [5]:
# keep top k most important features for further analysis, and modelling
k = 10

In [6]:
X_NR = mean_freq_per_sbj('task2', 'alpha', 'avg')
Y_NR = np.zeros((X_NR.shape[0], 1))

X_AR = mean_freq_per_sbj('task3', 'alpha', 'avg')
Y_AR = np.ones((X_AR.shape[0], 1))

X, y = np.vstack((X_NR, X_AR)), np.vstack((Y_NR, Y_AR))
X, y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
y_train, y_test = y_train.ravel(), y_test.ravel()

In [9]:
alpha_feature_weights = clf_fit(X_train, X_test, y_train, y_test, 'RandomForest')

np.savetxt('eeg_feature_extraction\\alpha_features_TRT.txt', np.argsort(alpha_feature_weights)[::-1][:k])

0.8986657050126217


In [10]:
X_NR = mean_freq_per_sbj('task2', 'theta', 'avg')
Y_NR = np.zeros((X_NR.shape[0], 1))

X_AR = mean_freq_per_sbj('task3', 'theta', 'avg')
Y_AR = np.ones((X_AR.shape[0], 1))

X, y = np.vstack((X_NR, X_AR)), np.vstack((Y_NR, Y_AR))
X, y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
y_train, y_test = y_train.ravel(), y_test.ravel()

In [11]:
theta_feature_weights = clf_fit(X_train, X_test, y_train, y_test, 'RandomForest')

np.savetxt('eeg_feature_extraction\\theta_features_TRT.txt', np.argsort(theta_feature_weights)[::-1][:k])

0.9065993508835196


In [12]:
X_NR = mean_freq_per_sbj('task2', 'beta', 'avg')
Y_NR = np.zeros((X_NR.shape[0], 1))

X_AR = mean_freq_per_sbj('task3', 'beta', 'avg')
Y_AR = np.ones((X_AR.shape[0], 1))

X, y = np.vstack((X_NR, X_AR)), np.vstack((Y_NR, Y_AR))
X, y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
y_train, y_test = y_train.ravel(), y_test.ravel()

In [13]:
beta_feature_weights = clf_fit(X_train, X_test, y_train, y_test, 'RandomForest')

np.savetxt('eeg_feature_extraction\\beta_features_TRT.txt', np.argsort(beta_feature_weights)[::-1][:k])

0.9228272628921745


In [14]:
X_NR = mean_freq_per_sbj('task2', 'gamma', 'avg')
Y_NR = np.zeros((X_NR.shape[0], 1))

X_AR = mean_freq_per_sbj('task3', 'gamma', 'avg')
Y_AR = np.ones((X_AR.shape[0], 1))

X, y = np.vstack((X_NR, X_AR)), np.vstack((Y_NR, Y_AR))
X, y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
y_train, y_test = y_train.ravel(), y_test.ravel()

In [15]:
gamma_feature_weights = clf_fit(X_train, X_test, y_train, y_test, 'RandomForest')

np.savetxt('eeg_feature_extraction\\gamma_features_TRT.txt', np.argsort(gamma_feature_weights)[::-1][:k])

0.9275153263613415


In [16]:
np.argsort(gamma_feature_weights)[::-1][:k]

array([ 30,  19,  99,  94,  35, 100,  36,  44,  40,   5], dtype=int64)

In [17]:
gamma_feature_weights[np.argsort(gamma_feature_weights)[::-1][:k]]

array([0.12328948, 0.10460418, 0.10108043, 0.0762451 , 0.06850493,
       0.03978786, 0.03777831, 0.03049938, 0.02646272, 0.02389829])